# **차량 공유업체의 차량 파손 여부 분류하기**

## 0.미션

* 1) 미션1 : Data Preprocessing
    - **과제 수행 목표**
        - 본인의 구글 드라이브에 모델링 수행을 위해 적절한 폴더 및 파일로 **일관성 있게 정리**해야 합니다.
        - 제공된 데이터 : Car_Images.zip
            * Car_Images : 차량의 정상/파손 이미지 무작위 수집

* 2) 미션2 : CNN 모델링
    - **과제 수행 목표**
        - Tensorflow Keras를 이용하여 모델을 3개 이상 생성하세요.
            - 모델 구조와 파라미터는 자유롭게 구성하세요.
            - 단, 세부 목차에서 명시한 부분은 지켜주세요.

* 3) 미션3 : Data Argumentation & Transfer Learning
    - **과제 수행 목표**
        - 성능 개선을 위해 다음의 두가지를 시도하세요.
            * Data Augmentation을 적용하세요.(Image Generator)
            * Transfer Learning(VGG16)


## 1.환경설정 

### (1) 데이터셋 폴더 생성
- **세부요구사항**
    - C드라이브에 Datasets라는 폴더를 만드세요.
        - 구글드라이브를 사용하는경우 드라이브 첫 화면에 Datasets 라는 폴더를 만드세요. ('/content/drive/MyDrive/Datasets/')
    - 해당 폴더 안에 Car_Images.zip 파일을 넣으세요.

* 구글 Colab을 이용하는 경우

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 데이터셋 불러오기 
- **세부요구사항**
    - Car_Images.zip 파일을 C:/Datasets/ 경로에 압축 해제합니다.
    - zipfile 모듈을 이용하거나 다른 방식을 사용해도 됩니다.
        - 참고 자료 : [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 폴더구조(로컬)
        * C:/Datasets/ : 압축파일
        * C:/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 폴더구조(구글드라이브브)
        * /content/drive/MyDrive/Datasets/ : 압축파일
        * /content/drive/MyDrive/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 압축을 해제하면 다음과 같은 두 하위 폴더가 생성됩니다.
        * normal, abnormal : 각 폴더에는 이미지들이 있습니다.
        * 이후 단계에서 해당 경로로 부터 validation, test 셋을 추출하게 됩니다.
        

In [2]:
import zipfile

In [3]:
# 압축파일 경로
# 구글 드라이브인 경우 경로에 맞게 지정하세요.
dataset_path  = '/content/drive/MyDrive/Datasets/'
# dataset_path = 'C:/Datasets/'

file_path = dataset_path + 'Car_Images.zip'

In [7]:
# 압축 해제
data = zipfile.ZipFile(file_path)
data.extractall('/content/drive/MyDrive/Datasets/Car_Images_train')

### (3) 이미지 저장을 위한 폴더 생성
- **세부요구사항**
    - train, validation, test 을 위해 각각 하위 폴더 normal과 abnormal를 준비합니다.
        - train
            * 정상 이미지 저장소 : C:/Datasets/Car_Images_train/normal/ 
                * 구글드라이브 :   /content/drive/MyDrive/Datasets/Car_Images_train/normal/
            * 파손 이미지 저장소 : C:/Datasets/Car_Images_train/abnormal/
                * 구글드라이브 : /content/drive/MyDrive/Datasets/Car_Images_train/abnormal/
        - val, test 역시 동일한 구조로 생성합니다.
    - 직접 탐색기에서 폴더를 생성할 수도 있고, os 모듈을 이용하여 코드로 작성할 수도 있습니다.
        - 참고 자료 : [os document](https://docs.python.org/3/library/os.html)

In [4]:
import os

# 각각 경로 지정
test_path = '/content/drive/MyDrive/Datasets/Car_Images_test/'
val_path = '/content/drive/MyDrive/Datasets/Car_Images_val/'

In [15]:
train_path = dataset_path+'Car_Images_train/'
os.mkdir(train_path)
os.mkdir(train_path+'normal')
os.mkdir(train_path+'abnormal')

In [10]:
# train 폴더는 압축을 해제하면서 이미 생성 되어 있습니다.

# test 폴더 만들기 os.mkdir()
os.mkdir(test_path)
os.mkdir(test_path+'normal')
os.mkdir(test_path+'abnormal')

# validation 폴더 만들기
os.mkdir(val_path)
os.mkdir(val_path+'normal')
os.mkdir(val_path+'abnormal')


## 2.데이터 전처리

### (1) 데이터 분할 : Training set | Validation set | Test set 생성
- **세부요구사항**
    - Training set, Validation set, Test set을 만듭니다.
        * size
            * test : 전체에서 20%를 추출합니다.
            * validation : test를 떼어낸 나머지에서 다시 20%를 추출합니다.
        * 데이터는 랜덤하게 추출해야 합니다.
            - random, shutil 모듈을 이용하여 랜덤하게 추출할 수 있습니다.
                - [random document](https://docs.python.org/3/library/random.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
            * 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

#### 1) test, validation 크기를 지정

In [5]:
import random, shutil

In [6]:
tr_n_path = '/content/drive/MyDrive/Datasets/Car_Images_train/normal'
tr_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_train/abnormal'

In [7]:
# 전체 이미지 개수를 확인합니다.
len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path))

(302, 303)

In [8]:
# test 사이즈 : 전체 이미지의 20%
te_data_num = [round(len(os.listdir(tr_n_path))*0.2), round(len(os.listdir(tr_ab_path))*0.2)]
print('test, norm, ab', te_data_num)

# validation 사이즈 : test를 제외한 나머지 중에서 20%
# 20퍼 안해도 됨
val_data_num = [ round((len(os.listdir(tr_n_path))-te_data_num[0])*0.2) , round((len(os.listdir(tr_n_path))-te_data_num[1])*0.2) ]
print('val, norm, ab', val_data_num)

# train 사이즈
train_data_num = [len(os.listdir(tr_n_path)) - te_data_num[0] - val_data_num[0],
                  len(os.listdir(tr_ab_path))- te_data_num[1] - val_data_num[1]]
print('train, norm, ab', train_data_num)

norm, ab [60, 61]
norm, ab [48, 48]
norm, ab [194, 194]


#### 2) test 셋 추출

In [9]:
import glob
random.seed(2023)

train_norm = glob.glob(tr_n_path + '/*.*')
train_ab = glob.glob(tr_ab_path + '/*.*')

test_norm = random.sample(train_norm, te_data_num[0])
test_ab = random.sample(train_ab, te_data_num[1])

In [44]:
test_norm[0]

'/content/drive/MyDrive/Datasets/Car_Images_train/normal/DALLíñE 2023-03-11 14.13.34 - part of a car.png'

In [48]:
# os 라이브러리 활용 연습
img = Image.open(test_norm[0])
title, ext = os.path.splitext(test_norm[0])
basename = os.path.basename(test_norm[0])
print(title)
print(ext)
print(basename)

/content/drive/MyDrive/Datasets/Car_Images_train/normal/DALLíñE 2023-03-11 14.13.34 - part of a car
.png
DALLíñE 2023-03-11 14.13.34 - part of a car.png


In [50]:
# test normal 데이터 저장
from PIL import Image

for f in test_norm:
    img = Image.open(f)

    base_name = os.path.basename(f)
    img.save(test_path + 'normal/' + base_name)

In [51]:
# test abnormal 데이터 저장
from PIL import Image

for f in test_ab:
    img = Image.open(f)

    base_name = os.path.basename(f)
    img.save(test_path + 'abnormal/' + base_name)

In [37]:
# 추출 후 이미지 갯수 확인
print(len(test_norm), len(test_ab))

60 61


In [10]:
# train 데이터에서 test 데이터 빼주기
train_norm = list(set(train_norm) - set(test_norm))
train_ab = list(set(train_ab) - set(test_ab))
print(len(train_norm), len(train_ab))

242 242


#### 3) validation 셋 추출

In [11]:
random.seed(2023)

val_norm = random.sample(train_norm, val_data_num[0])
val_ab = random.sample(train_ab, val_data_num[1])

# train 데이터에서 validation 데이터 빼주기
train_norm = list(set(train_norm) - set(val_norm))
train_ab = list(set(train_ab) - set(val_ab))
print(len(train_norm), len(train_ab))

194 194


In [13]:
from PIL import Image

# validation normal 데이터 저장 
for f in val_norm:
    img = Image.open(f)

    base_name = os.path.basename(f)
    img.save(val_path + 'normal/' + base_name)

# validation abnormal 데이터 저장
for f in val_ab:
    img = Image.open(f)

    base_name = os.path.basename(f)
    img.save(val_path + 'abnormal/' + base_name)

In [18]:
from PIL import Image

train_path = '/content/drive/MyDrive/Datasets/Car_Images_train_split/'

# train normal 데이터 저장 
for f in train_norm:
    img = Image.open(f)

    base_name = os.path.basename(f)
    img.save(train_path + 'normal/' + base_name)

# train abnormal 데이터 저장
for f in train_ab:
    img = Image.open(f)

    base_name = os.path.basename(f)
    img.save(train_path + 'abnormal/' + base_name)

In [19]:
# 추출 후 이미지 갯수 확인
print(len(train_norm), len(train_ab))
print(len(test_norm), len(test_ab))
print(len(val_norm), len(val_ab))

194 194
60 61
48 48


In [20]:
tr_n_path = '/content/drive/MyDrive/Datasets/Car_Images_train_split/normal'
tr_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_train_split/abnormal'
te_n_path = '/content/drive/MyDrive/Datasets/Car_Images_test/normal'
te_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_test/abnormal'
va_n_path = '/content/drive/MyDrive/Datasets/Car_Images_val/normal'
va_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_val/abnormal'

print(len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path)))
print(len(os.listdir(te_n_path)) , len(os.listdir(te_ab_path)))
print(len(os.listdir(va_n_path)) , len(os.listdir(va_ab_path)))

194 194
60 61
48 48


### (2) 데이터 복사 및 이동
- **세부요구사항**
    - 분할된 데이터를 복사 이동합니다.
        - 새로운 폴더에 저장하는 데이터로 "3.모델링I"에서 사용합니다.
        - 기존 폴더는 "4.모델링II > (1) Data Augmentation"에서 사용합니다.
    - Training set | Validation set | Test set의 데이터를 **새로운 폴더**에 복사하세요.
        - 새로운 폴더 명
            * copy_images/trainset
            * copy_images/validset
            * copy_images/testset
        - 새로운 폴더에는 normal, abnormal 파일 모두를 복사합니다. 
            * 파일을 구분하기 위해 abnormal 파일들은 파일명 앞에 접두사 'ab_'를 붙입시다.
        - os, shutil 모듈을 활용하세요.

#### 1) abnormal 파일 복사

* 복사하기 : shutil.copytree()

In [35]:
dst = '/content/drive/MyDrive/Datasets/copy_images/'

In [22]:

shutil.copytree(tr_ab_path, dst+'trainset')
shutil.copytree(te_ab_path, dst+'testset')
shutil.copytree(va_ab_path, dst+'validset')


'/content/drive/MyDrive/Datasets/copy_images/validset'

* abnormal 이미지 이름의 접두어 "ab_" 붙이기 : os.rename

In [37]:
import os
train_copy_data = os.listdir(dst+'trainset')
test_copy_data = os.listdir(dst+'testset')
val_copy_data = os.listdir(dst+'validset')

print(train_copy_data[0])
# DALLíñE 2023-03-11 00.14.25 - a part of car without scratch.png

for f in train_copy_data:
    src = os.path.join(dst+'trainset', f)
    new = os.path.join(dst+'trainset', 'ab_'+f)
    os.rename(src, new)

DALLíñE 2023-03-11 00.14.25 - a part of car without scratch.png


In [38]:
for f in test_copy_data:
    src = os.path.join(dst+'testset', f)
    new = os.path.join(dst+'testset', 'ab_'+f)
    os.rename(src, new)

In [39]:
for f in val_copy_data:
    src = os.path.join(dst+'validset', f)
    new = os.path.join(dst+'validset', 'ab_'+f)
    os.rename(src, new)

#### 2) normal 파일 복사

In [41]:
from distutils.dir_util import copy_tree
copy_tree(tr_n_path, dst+'trainset')
copy_tree(te_n_path, dst+'testset')
copy_tree(va_n_path, dst+'validset')

['/content/drive/MyDrive/Datasets/copy_images/validset/DALLíñE 2023-03-11 14.30.01 - part of a car.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLíñE 2023-03-10 23.31.15 - photo of a part of car without blemish.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLíñE 2023-03-11 14.15.38 - part of a car.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLíñE 2023-03-11 14.12.36 - part of a car.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLíñE 2023-03-11 14.32.32 - part of a car.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLíñE 2023-03-11 14.22.56 - part of a car.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLíñE 2023-03-11 14.16.08 - part of a car.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLíñE 2023-03-11 00.32.41 - photo of a part of car without blemish.png',
 '/content/drive/MyDrive/Datasets/copy_images/validset/DALLíñE 2023-03-11 00.36.19 - photo o

* 데이터 갯수 조회

In [42]:
print(len(os.listdir(dataset_path+'copy_images/trainset/')))
print(len(os.listdir(dataset_path+'copy_images/validset/')))
print(len(os.listdir(dataset_path+'copy_images/testset/')))

388
96
121


## 3.모델링 I
* **세부요구사항**
    * 모델링을 위한 데이터 구조 만들기
        * x : 이미지를 array로 변환합니다.
        * y : 이미지 갯수만큼 normal - 0, abnormal - 1 로 array를 만듭니다.
    * 모델을 최소 3개 이상 만들고 성능을 비교합니다.
        * 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
        * 전처리 과정에서 생성한 Validation set을 적절하게 사용하세요.
        * Early Stopping을 반드시 사용하세요.
            * 최적의 가중치를 모델에 적용하세요.

In [43]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

### (1) X : image to array
- **세부요구사항**
    * 모델링을 위해서는 np.array 형태로 데이터셋을 만들어야 합니다.
    * Training set / Validation set / Test set의 X는 이미지 형태로 되어있습니다. 
    * 이미지 파일을 불러와 train, valid, test 각각 array 형태로 변환해 봅시다.
        * 각 폴더로 부터 이미지 목록을 만들고
        * 이미지 한장씩 적절한 크기로 로딩하여 (keras.utils.load_img)
            * 이미지가 너무 크면 학습시간이 많이 걸리고, 메모리 부족현상이 발생될 수 있습니다.
            * 이미지 크기를 280 * 280 * 3 이내의 크기를 설정하여 로딩하시오.
            * array로 변환 (keras.utils.img_to_array, np.expand_dims)
        * 데이터셋에 추가합니다.(데이터셋도 array)

#### 1) 이미지 목록 만들기
* train, validation, test 폴더로 부터 이미지 목록을 생성합니다.

In [44]:
# 이미지 목록 저장
img_train_list = os.listdir(dataset_path+'copy_images/trainset/')
img_valid_list = os.listdir(dataset_path+'copy_images/validset/')
img_test_list = os.listdir(dataset_path+'copy_images/testset/')

In [45]:
# 메모리, 처리시간을 위해서 이미지 크기 조정
img_size = 256 ## 사이즈 조정 가능

#### 2) 이미지들을 배열 데이터셋으로 만들기

In [56]:
import numpy as np
from PIL import Image

dst = dataset_path+'copy_images/'
train_x = []
test_x = []
val_x = []

for f in img_train_list:
    img = Image.open(dst+'trainset/'+f)
    x = np.array(img)
    train_x.append(x)

for f in img_test_list:
    img = Image.open(dst+'testset/'+f)
    x = np.array(img)
    test_x.append(x)

for f in img_valid_list:
    img = Image.open(dst+'validset/'+f)
    x = np.array(img)
    val_x.append(x)

In [58]:
print(train_x[0][0][0])
print(test_x[0][0][0])
print(val_x[0][0][0])

[195 196 196]
[198 199 205]
[204 218 230]


In [ ]:
import tensorflow as tf
import keras
from keras.preprocessing.image import img_to_array, load_img, array_to_img

img = load_img(img_path, target_size=(img_size, img_size))
img_tensor = img_to_array(img)

img_tensor = np.expand_dims(img_tensor, axis=0)


### (2) y : 클래스 만들기
- **세부요구사항**
    - Training set / Validation set / Test set의 y를 생성합니다.
        - 각각 normal, abnormal 데이터의 갯수를 다시 확인하고
        - normal을 0, abnormal을 1로 지정합니다.

In [46]:
# 데이터 갯수 확인
print( len(img_train_list) )
print( len([val for val in img_train_list if val.startswith('ab_')]) )
print('---')
print( len(img_valid_list) )
print( len([val for val in img_valid_list if val.startswith('ab_')]) )
print('---')
print( len(img_test_list) )
print( len([val for val in img_test_list if val.startswith('ab_')]) )

388
194
---
96
48
---
121
61


* y_train, y_valid, y_test 만들기
    * normal, abnormal 데이터의 갯수를 다시 확인하고 normal을 0, abnormal을 1로 지정합니다.

In [68]:
y_train = []
y_valid = []
y_test = []

for val in img_train_list:
    if val.startswith('ab_'):
        y_train.append(1)
    else:
        y_train.append(0)

for val in img_test_list:
    if val.startswith('ab_'):
        y_test.append(1)
    else:
        y_test.append(0)

for val in img_valid_list:
    if val.startswith('ab_'):
        y_valid.append(1)
    else:
        y_valid.append(0)

In [67]:
print(img_train_list[:5])
print(y_train[:5])
print(img_train_list[-5:])
print(y_train[-5:])

['ab_DALLíñE 2023-03-11 00.14.25 - a part of car without scratch.png', 'ab_DALLíñE 2023-03-11 17.17.32 - scratched car.png', 'ab_DALLíñE 2023-03-11 17.22.20 - scratched car.png', 'ab_DALLíñE 2023-03-11 17.18.25 - scratched car.png', 'ab_DALLíñE 2023-03-11 01.22.25 - scratched car.png']
[1, 1, 1, 1, 1]
['DALLíñE 2023-03-10 23.39.17 - photo of a part of car without blemish.png', 'DALLíñE 2023-03-11 14.29.25 - part of a car.png', 'DALLíñE 2023-03-11 14.26.16 - part of a car.png', 'DALLíñE 2023-03-10 23.32.14 - photo of a part of car without blemish.png', 'DALLíñE 2023-03-11 01.36.44 - a part of a car.png']
[0, 0, 0, 0, 0]


### (3) 모델1
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random as rd
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras

In [75]:
train_x.shape

AttributeError: ignored

In [ ]:
# 1. 세션 클리어
keras.backend.clear_session()

# 2. 모델 선언
model = keras.models.Sequential()

# 3. 블록 조립
# 인풋
model.add(keras.layers.Input(shape=(32,32,3)))
# Conv 필터 32개, 사이즈 3,3, same padding
model.add(keras.layers.Conv2D(filters=32,
                              kernel_size = (3,3),
                              padding= 'same',
                              strides=(1,1),
                              activation='relu'))
# Conv 필터 32개, 사이즈 3,3, same padding
model.add(keras.layers.Conv2D(filters=32,
                              kernel_size = (3,3),
                              padding= 'same',
                              strides=(1,1),
                              activation='relu'))
# BatchNorm
model.add(keras.layers.BatchNormalization())
# Maxpooling 사이즈 2,2 스트라이드 2,2
model.add(keras.layers.MaxPool2D(pool_size=(2,2), # pooling filter 의 가로 세로 사이즈
                                 strides=(2,2),))  # 어떻게 이동할 것인지
# Dropout 25%
model.add(keras.layers.Dropout(0.25))
# Conv 필터 64개, 사이즈 3,3, same padding
model.add(keras.layers.Conv2D(filters=64,
                              kernel_size = (3,3),
                              padding= 'same',
                              strides=(1,1),
                              activation='relu'))
# Conv 필터 64개, 사이즈 3,3, same padding
model.add(keras.layers.Conv2D(filters=64,
                              kernel_size = (3,3),
                              padding= 'same',
                              strides=(1,1),
                              activation='relu'))
# BatchNorm
model.add(keras.layers.BatchNormalization())
# Maxpooling 사이즈 2,2 스트라이드 2,2
model.add(keras.layers.MaxPool2D(pool_size=(2,2), # pooling filter 의 가로 세로 사이즈
                                 strides=(2,2),))  # 어떻게 이동할 것인지
# Dropout 25%
model.add(keras.layers.Dropout(0.25))
# Flatten
model.add(keras.layers.Flatten())
# Fully Connected layer, 노드 1024개
model.add(keras.layers.Dense(1024, activation='relu'))
# BatchNorm
model.add(keras.layers.BatchNormalization())
# Dropout 35%
model.add(keras.layers.Dropout(0.35))
# output layer
model.add(keras.layers.Dense(10, activation='softmax'))

# 4. 컴파일
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
              optimizer='adam')

# 요약
model.summary()

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

### (4) 모델2
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

### (5) 모델3
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

## 4.모델링 II
* **세부요구사항**
    - 성능을 높이기 위해서 다음의 두가지를 시도해 봅시다.
        - Data Augmentation을 통해 데이터를 증가 시킵니다.
            - ImageDataGenerator를 사용합니다.
        - 사전 학습된 모델(Transfer Learning)을 가져다 사용해 봅시다.
            - VGG16(이미지넷)을 사용해 봅시다.

### (1) Data Augmentation
- **세부요구사항**
    * 모델 학습에 이용할 이미지 데이터를 증강시키세요.
    * Keras의 ImageDataGenerator를 이용
        - [ImageDataGenerator document](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

    * image generator를 이용하여 학습
        * 모델 구조는 이미 생성한 1,2,3 중 하나를 선택하여 학습


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_size = 280 ## 사이즈 조정 가능

train_path = dataset_path+'Car_Images_train/'
valid_path = dataset_path+'Car_Images_valid/'

#### 1) ImageGenerator 생성
* ImageDataGenerator 함수 사용
    * 주요 옵션
        * rotation_range: 무작위 회전을 적용할 각도 범위
        * zoom_range: 무작위 줌을 적용할 범위 [1-zoom_range, 1+zoom_range]
        * horizontal_flip: 무작위 좌우반전을 적용할지 여부
        * vertical_flip: 무작위 상하반전을 적용할지 여부
        * rescale: 텐서의 모든 값을 rescale 값으로 나누어줌 (이 경우에는 255로 나누어서 0~1사이의 값으로 변경)

In [ ]:
train_datagen = 

valid_datagen = 


#### 2) 경로로 부터 이미지 불러 올 준비
* .flow_from_directory 이용
    * 디렉토리에서 이미지를 가져와서 데이터 증강을 적용하고 batch 단위로 제공하는 generator를 생성합니다.
    * 이미지를 불러올 때 target_size로 크기를 맞추고, 
    * class_mode로 이진 분류(binary)를 수행하도록 지정합니다.


In [ ]:
train_generator = 

valid_generator = 


#### 3) 학습
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 train_generator 이용. 
    - validation_data = valid_generator 지정
    - Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

* 구조 설계

* 학습
    * EarlyStopping 설정하기
    * 학습 데이터에 train_generator, validation_data=valid_generator 사용

#### 4) 성능 평가
* 평가는 confusion_matrix, classification_report 활용

### (2) Transfer Learning
- **세부요구사항**
    * VGG16 모델은 1000개의 클래스를 분류하는 데 사용된 ImageNet 데이터셋을 기반으로 사전 학습된 가중치를 가지고 있습니다. 
        * 따라서 이 모델은 이미지 분류 문제에 대한 높은 성능을 보입니다.
        * 이 모델은 보통 전이학습(transfer learning)에서 기본적으로 사용되며, 특히 대규모 데이터셋이 없을 때는 기본 모델로 사용되어 fine-tuning을 수행합니다.
    * VGG16 함수로 부터 base_model 저장


In [ ]:
from tensorflow.keras.applications import VGG16

#### 1) VGG16 불러와서 저장하기
* include_top=False로 설정하여 분류기를 제외하고 미리 학습된 가중치 imagenet을 로드합니다.
* .trainable을 True로 설정하여 모델의 모든 레이어들이 fine-tuning에 대해 업데이트되도록 합니다.


In [ ]:
base_model = VGG16(                 )




#### 2) VGG16과 연결한 구조 설계
* VGG16을 불러와서 Flatten, Dense 등으로 레이어 연결하기

#### 3) 학습
- **세부요구사항**
    - 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
    - 데이터
        * Image Generator를 연결하거나
        * 기존 train, validation 셋을 이용해도 됩니다.
        - Early Stopping을 반드시 사용하세요.
        - 최적의 가중치를 모델에 적용하세요.

#### 4) 성능 평가